In [36]:
# %pip install datasets
# %pip install Pillow
# %pip install numpy
# %pip install matplotlib
# %pip install import_ipynb
# %pip install tensorflow
# %pip install tensorflow.keras


# # !pip install torch
# # !pip install torchvision
# # !pip install scikit-image
# # !pip install scipy
# %pip install easyocr

# # library from etl
# %pip install datasets
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install json

In [ ]:
import easyocr
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
# from tensorflow.keras import layers
import json

In [38]:
# from google.colab import userdata

# # Retrieve the GitHub token from userdata
# myGithub = userdata.get('githubKey')

# !git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice-image-processing
# !git -C invoice-image-processing pull



In [39]:
# import sys
# import os

# sys.path.append('/content/invoice-image-processing')
# print(os.listdir('/content/invoice-image-processing'))

# !jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

Image Classification

In [ ]:
from data_etl import DatasetReceipt

In [ ]:
# Inisialisasi dataset
dataset_local = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
dataset_inter = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")

In [ ]:
from keras.applications.ResNet50V2 import ResNet50V2

# keras.applications.ResNet50V2(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     name="resnet50v2",
# )

pre_trained_model = ResNet50V2(input_shape = (600, 600, 3),
                                include_top = False,
                                pooling='max'
                                )

pre_trained_model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load Dataset (Gunakan DatasetReceipt class yang sudah dibuat sebelumnya)
train_dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
val_dataset = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")

# Function to generate batches
def dataset_generator(dataset, batch_size=16):
    while True:
        images, labels = [], []
        for i in range(batch_size):
            sample = dataset[i]
            images.append(sample["image"])
            labels.append(sample["label"])  # Ubah ke format numerik jika perlu

        yield np.array(images), np.array(labels)

# Pretrained Model (Feature Extractor)
base_model = ResNet50V2(input_shape=(600, 600, 3), include_top=False, pooling='max')
base_model.trainable = False  # Freeze pretrained layers

# Tambahkan Custom Layers untuk Bounding Box + Label Classification
inputs = Input(shape=(600, 600, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output untuk bounding box (4 nilai: x_min, y_min, x_max, y_max)
bbox_output = Dense(4, activation='linear', name="bounding_box")(x)

# Output untuk klasifikasi teks (misalnya 5 kelas)
class_output = Dense(5, activation='softmax', name="class_label")(x)

# Gabungkan menjadi satu model
model = Model(inputs, outputs=[bbox_output, class_output])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        "bounding_box": "mse",  # Mean Squared Error untuk bounding box
        "class_label": "categorical_crossentropy",  # Klasifikasi teks
    },
    metrics={"bounding_box": "mae", "class_label": "accuracy"}
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

checkpoint_path = "/content/drive/MyDrive/dataset/model/model_checkpoint_resnet50v2.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_best_only=False)

In [ ]:
class Callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.95 and logs['val_accuracy'] > 95:
            print("\nAccuracy reached 95%! Stopping training.")
            self.model.stop_training = True

stop_callback = Callbacks()

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training_log.csv', separator=',', append=True)

In [ ]:
# Data Augmentation
train_generator = dataset_generator(train_dataset, batch_size=16)
val_generator = dataset_generator(val_dataset, batch_size=16)

# Train Model
model.fit(train_generator, 
          validation_data=val_generator,
          epochs=25,
          steps_per_epoch=1000,
          validation_steps=200,
          callbacks=[checkpoint_callback, csv_logger])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load training log
log_data = pd.read_csv("training_log.csv")

# Buat figure untuk dua grafik
plt.figure(figsize=(16, 6))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.title("Model Accuracy")
plt.plot(log_data["epoch"], log_data["bounding_box_mae"], label="Training MAE")
plt.plot(log_data["epoch"], log_data["val_bounding_box_mae"], label="Validation MAE")
plt.plot(log_data["epoch"], log_data["class_label_accuracy"], label="Training Accuracy")
plt.plot(log_data["epoch"], log_data["val_class_label_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy / MAE")
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.title("Model Loss")
plt.plot(log_data["epoch"], log_data["loss"], label="Training Loss")
plt.plot(log_data["epoch"], log_data["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
# # Save Model
# model.save("faster_rcnn_resnet50v2.h5")
# print("Model saved successfully!")

**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [ ]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text):
    text_lower = text.lower()

    if 'total' or 'amount' or 'jumlah' and 'price' or 'harga' in text_lower:
        print("This is a valid receipt.")
    else:
        print("This is not a valid receipt.")


def process_image(image):
    # Convert PIL Image to numpy array
    image_np = np.array(image)
    if image_np.dtype != np.uint8:
        image_np = (image_np * 255).astype(np.uint8)  # Normalisasi jika float64

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    is_receipt(extracted_text)

    print("Extracted Text:")
    print(extracted_text)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Fine Tuning

In [ ]:
# data_tuning_local = load_dataset("naver-clova-ix/cord-v2", split="train")
# data_tuning_inter = load_dataset("katanaml-org/invoices-donut-data-v1", split="train")
# data_tuning_inter[5]


{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2481x3508>,
 'ground_truth': '{"gt_parse": {"header": {"invoice_no": "42485588", "invoice_date": "03/15/2012", "seller": "Zuniga and Sons 455 Jones Trace Suite 479 Port Sarah, NM 29233", "client": "Knight-Brown 316 Garrett Valleys Lake John, LA 21710", "seller_tax_id": "923-88-8185", "client_tax_id": "970-99-7937", "iban": "GB47NOK145420514797164"}, "items": [{"item_desc": "3 Gal 12L Moonshine Still Stainless Steel Water Wine Alcohol Distiller Equipment", "item_qty": "1,00", "item_net_price": "99,92", "item_net_worth": "99,92", "item_vat": "10%", "item_gross_worth": "109,91"}, {"item_desc": "Custom wine glass", "item_qty": "5,00", "item_net_price": "19,00", "item_net_worth": "95,00", "item_vat": "10%", "item_gross_worth": "104,50"}], "summary": {"total_net_worth": "$ 194,92", "total_vat": "$ 19,49", "total_gross_worth": "214,41"}}}'}

In [ ]:
# import os
# import json

# def convert_dataset_local(dataset, output_file):
#     annotations = []

#     # Pastikan folder untuk menyimpan gambar ada
#     os.makedirs("dataset_images", exist_ok=True)

#     for i, data in enumerate(dataset):
#         image = data["image"]  # Objek gambar dari PIL
#         ground_truth = json.loads(data["ground_truth"])  # Parsing JSON string

#         # Simpan gambar ke folder agar bisa digunakan oleh EasyOCR
#         image_path = f"dataset_images/image_{i}.jpg"
#         image.save(image_path)  

#         # Ambil teks dan bounding box dari ground truth
#         text = ground_truth["gt_parse"]  # Semua teks dan informasi struk
#         words = []
#         for line in ground_truth["valid_line"]:  # Ambil setiap baris
#             for word in line["words"]:  # Ambil setiap kata dalam baris
#                 words.append({
#                     "text": word["text"],
#                     "quad": word["quad"],
#                     "row_id": word["row_id"],
#                     "category": line["category"],
#                     "group_id": line["group_id"]
#                 })

#         # Simpan dalam format yang sesuai untuk EasyOCR
#         annotations.append({
#             "image": image_path,
#             "text": text,
#             "words": words
#         })

#     # Simpan hasil dalam file JSON
#     with open(output_file, "w", encoding="utf-8") as f:
#         json.dump(annotations, f, indent=4, ensure_ascii=False)
        

# def convert_dataset_inter(dataset, output_file):
#     """
#     Mengonversi dataset faktur/invoice ke format JSON untuk EasyOCR.
#     """
#     annotations = []
    
#     for i, data in enumerate(dataset):
#         image = data["image"]  # PIL Image
#         ground_truth = json.loads(data["ground_truth"])["gt_parse"]  # Parse JSON
        
#         # Simpan gambar
#         image_path = f"dataset_images/invoice_{i}.jpg"
#         os.makedirs("dataset_images", exist_ok=True)
#         image.save(image_path)
        
#         words = []
#         items = ground_truth.get("items", [])  # Pastikan mengambil 'items' dengan default []
#         if items:
#             for item in items:
#                 words.append({
#                     "text": item.get("item_desc", "UNKNOWN_ITEM"),
#                     "quad": None,
#                     "row_id": None,
#                     "group_id": None
#                 })
        
#         annotations.append({
#             "image": image_path,
#             "text": ground_truth,  # Simpan seluruh teks yang diparsing
#             "bbox": words
#         })
    
#         # Pastikan folder Google Drive tujuan benar
#         drive_folder = "G:/My Drive/dataset"
#         output_file = os.path.join(drive_folder, "dataset_annotations.json")

#         # Simpan sebagai JSON
#         with open(output_file, "w", encoding="utf-8") as f:
#             json.dump(annotations, f, indent=4, ensure_ascii=False)

#         print(f"Dataset invoice berhasil disimpan ke {output_file}")


In [ ]:
# # Konversi dataset 1 dan dataset 2
# convert_dataset_local(data_tuning_local, "data_tuning_local.json")
# convert_dataset_inter(data_tuning_inter, "data_tuning_inter.json")

Dataset invoice berhasil disimpan ke data_tuning_inter.json


In [ ]:
# python train.py --batch_size 16 --num_epochs 10 --lr 0.001 --train_data data_tuning_local.json data_tuning_inter.json


POST-PROCESSING

Pre-Processing Hasil Text Extraction

Training BERT/NER